In [1]:
! pip install llama-index == 0.11.13 
! pip install llama-index-llms-ollama
! pip install llama-index-embeddings-ollama

zsh:1: = not found


In [2]:
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

In [3]:
import os
ollama_host = os.getenv("OLLAMA_API", "http://localhost:11434")
ollama_model = os.getenv("OLLAMA_MODEL_LLAMA", "llama3.2:latest")
logger.info(f"ollama_host: {ollama_host}")
logger.info(f"ollama_model: {ollama_model}")

INFO:__main__:ollama_host: http://localhost:11434
INFO:__main__:ollama_model: llama3.2:latest


In [38]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

llm = Ollama(model=ollama_model, base_url=ollama_host)

resp = llm.complete(prompt="Hello, world!")
logger.info(resp)

embedding = OllamaEmbedding(model_name=ollama_model, base_url=ollama_host)
Settings.embed_model = embedding
Settings.llm = llm

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/tju/.pyenv/versions/3.11.10/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/tju/.pyenv/versions/3.11.10/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/tju/.pyenv/versions/3.11.10/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started host='127.0.0.1' port=7890 local_address=None timeout=30.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x3042a9cd0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcor

RemoteProtocolError: Server disconnected without sending a response.

> Document Nodes

In [41]:
from llama_index.core import Document
from llama_index.core.schema import TextNode

doc = Document(text="This is a sample document text")
n1 = TextNode(text=doc.text[0: 16])
n2 = TextNode(text=doc.text[17: 30])
logger.info(n1)
logger.info(n2)


INFO:__main__:Node ID: a6fefb92-8387-4ae4-87e4-92624605c5ea
Text: This is a sample
INFO:__main__:Node ID: 440a2c9a-cdc0-432c-aa13-0cbc7bd1cd2d
Text: document text


> Document Splitter 

In [ ]:
from llama_index.core import Document
from llama_index.core.schema import TextNode 
from llama_index.core.node_parser import TokenTextSplitter
doc = Document(
    text=("this is sentence 1. this is sentence 2. " "this is sentence 3."),
    metadata={"author": "John Smith"}
    )

splitter = TokenTextSplitter(chunk_size=12, chunk_overlap=0, separator=" ")
nodes = splitter.get_nodes_from_documents([doc])
for node in nodes:
    print(f'node: {node}')
    print(f'metadata: {node.metadata}')


> Node Relationship
```
* Enable more contextual querying
* Allows tracking provenance
* Enables navigation through the nodes
* Supports the construction of knowledge graphs
* Improves the index structure
```

In [ ]:
from llama_index.core import Document
from llama_index.core.schema import TextNode, NodeRelationship
from llama_index.core.node_parser import TokenTextSplitter
doc = Document(
    text=("first sentence. second sentence."),
    metadata={"author": "John Smith"}
    )
n1 = TextNode(text="first sentence.", node_id=doc.doc_id)
n2 = TextNode(text="second sentence.", node_id=doc.doc_id)

n1.relationships[NodeRelationship.NEXT] = n2.node_id
n2.relationships[NodeRelationship.PREVIOUS] = n1.node_id

print(n1.relationships)
print(n2.relationships)


> Summary Index


In [ ]:
from llama_index.core.settings import Settings

from llama_index.core.schema import TextNode
from llama_index.core import SummaryIndex

nodes = [
  TextNode(text="Lionel Messi's hometown is Rosario."),
  TextNode(text="He was born on June 24, 1987.")
]
index = SummaryIndex(nodes)
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query(
    "What is Messi's hometown?"
)
print(response)

In [ ]:
! pip install llama-index-readers-wikipedia
! pip install wikipedia

> Reading wiki content


In [3]:
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core import Document, SummaryIndex
from llama_index.core.node_parser import SimpleNodeParser

reader = WikipediaReader()
docs = reader.load_data(pages=["https://en.wikipedia.org/wiki/Lionel_Messi"])
print(docs)
parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(docs)
index = SummaryIndex(nodes)
query_engine = index.as_query_engine(llm=llm)
print("Ask me anything about Lionel Messi!")

response = query_engine.query("What is Messi's hometown?")
print(response)


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): en.wikipedia.org:80
